<a href="https://colab.research.google.com/github/deekshahs1817/Infosys_SpringBoard_All_milestones-1-4-/blob/main/dee_milestone_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install streamlit easyocr opencv-python-headless fuzzywuzzy python-Levenshtein pandas pillow matplotlib spacy pyngrok
!python -m spacy download en_core_web_sm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 37.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 65.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.3/153.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 63.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.7/180.7 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 978.2/978.2 kB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.6/300.6 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 62.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 110.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the

In [ ]:
!pip install streamlit pyngrok opencv-python-headless pillow pytesseract numpy sentence-transformers
!apt-get update -qq && apt-get install -y -qq tesseract-ocr


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [ ]:
%%writefile app.py
# app.py — AI-Powered KYC Fraud Detection System (Final Version)
# Author: Deeksha H S | Streamlit + EasyOCR + spaCy + SQLite + Analytics

import streamlit as st
import sqlite3, hashlib, easyocr, cv2, numpy as np, pandas as pd, re, io, os, spacy
from datetime import datetime
from fuzzywuzzy import fuzz
from PIL import Image, ImageChops
import matplotlib.pyplot as plt

# ---------------- CONFIG ----------------
st.set_page_config(page_title="🛡️ AI KYC Verification", page_icon="🔍", layout="wide")
DB_PATH = "kyc_final_admin_analytics.db"

# ---------------- INIT DATABASE ----------------
def init_db():
    conn = sqlite3.connect(DB_PATH)
    c = conn.cursor()
    c.execute("""CREATE TABLE IF NOT EXISTS users(
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        username TEXT UNIQUE, password_hash TEXT, role TEXT)""")
    c.execute("""CREATE TABLE IF NOT EXISTS verifications(
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        username TEXT, document_type TEXT, name_claimed TEXT,
        extracted_number TEXT, extracted_name TEXT,
        ela_score REAL, blur_score REAL, name_score REAL,
        fraud_score REAL, risk_level TEXT, ocr_text TEXT,
        approval_status TEXT, created_at TEXT)""")
    conn.commit()
    conn.close()
init_db()

# ---------------- HELPERS ----------------
def sha256_text(s): return hashlib.sha256(s.encode()).hexdigest() if s else ""
def hash_pw(p): return hashlib.sha256(p.encode()).hexdigest()
def mask_number(num):
    if not num: return ""
    digits = re.sub(r'\s+', '', num)
    return "XXXX-XXXX-" + digits[-4:] if len(digits) >= 12 else digits

# ---------------- LOAD MODELS ----------------
@st.cache_resource
def load_ocr(): return easyocr.Reader(['en'], gpu=False)
@st.cache_resource
def load_nlp():
    try: return spacy.load("en_core_web_sm")
    except:
        import subprocess; subprocess.run(["python","-m","spacy","download","en_core_web_sm"])
        return spacy.load("en_core_web_sm")

reader = load_ocr()
nlp = load_nlp()

# ---------------- OCR / IMAGE FUNCTIONS ----------------
def decode_img(b): return cv2.imdecode(np.frombuffer(b, np.uint8), cv2.IMREAD_COLOR)
def ocr_text(b):
    try: img = decode_img(b); return " ".join(reader.readtext(img, detail=0))
    except: return ""
def ela_score(b):
    try:
        im = Image.open(io.BytesIO(b)).convert("RGB")
        buf = io.BytesIO(); im.save(buf, format="JPEG", quality=90)
        diff = ImageChops.difference(im, Image.open(buf).convert("RGB"))
        return float(np.mean(np.array(diff)))
    except: return 0
def blur_score(b):
    img = decode_img(b)
    if img is None: return 0
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return float(cv2.Laplacian(gray, cv2.CV_64F).var())

# ---------------- EXTRACTORS ----------------
def extract_aadhaar(t): m=re.search(r'\b(\d{4}\s?\d{4}\s?\d{4})\b', t); return (m.group(1) if m else "")
def extract_pan(t): m=re.search(r'\b([A-Z]{5}[0-9]{4}[A-Z])\b', t.upper()); return (m.group(1) if m else "")
def extract_license(t): m=re.search(r'\b([A-Z]{2}\d{2}\s?\d{4,11})\b', t.upper()); return (m.group(1) if m else "")

# ---------------- FRAUD SCORING ----------------
def name_similarity(n1, n2):
    if not n1 or not n2: return 0
    fuzzy = fuzz.token_set_ratio(n1.lower(), n2.lower())
    try: spacy_sim = nlp(n1).similarity(nlp(n2))*100
    except: spacy_sim = 0
    return (fuzzy + spacy_sim)/2

def fraud_score(valid, ns, ela, blur):
    score = 0
    if valid: score += 40
    if ns >= 80: score += 30
    elif ns >= 60: score += 15
    if ela > 15: score -= 15
    if blur < 100: score -= 10
    return max(0, min(100, score))

def risk_label(s):
    if s >= 80: return "Low Risk 🟢"
    elif s >= 50: return "Medium Risk 🟠"
    else: return "High Risk 🔴"

# ---------------- DATABASE OPS ----------------
def save_record(rec):
    conn = sqlite3.connect(DB_PATH)
    c = conn.cursor()
    c.execute("""INSERT INTO verifications(
        username, document_type, name_claimed, extracted_number, extracted_name,
        ela_score, blur_score, name_score, fraud_score, risk_level, ocr_text, approval_status, created_at)
        VALUES(?,?,?,?,?,?,?,?,?,?,?,?,?)""",
        (rec["username"], rec["document_type"], rec["name_claimed"], rec["extracted_number"], rec["extracted_name"],
         rec["ela_score"], rec["blur_score"], rec["name_score"], rec["fraud_score"], rec["risk_level"],
         rec["ocr_text"], "Pending", datetime.now().strftime("%Y-%m-%d %H:%M:%S")))
    conn.commit(); conn.close()

def update_status(id, status):
    conn=sqlite3.connect(DB_PATH); c=conn.cursor()
    c.execute("UPDATE verifications SET approval_status=? WHERE id=?",(status,id))
    conn.commit(); conn.close()

# ---------------- AUTH ----------------
def register_user(u,p,r):
    conn=sqlite3.connect(DB_PATH);c=conn.cursor()
    try: c.execute("INSERT INTO users(username,password_hash,role) VALUES(?,?,?)",(u,hash_pw(p),r));conn.commit();return True
    except sqlite3.IntegrityError:return False
    finally:conn.close()

def login_user(u,p): conn=sqlite3.connect(DB_PATH);c=conn.cursor();c.execute("SELECT * FROM users WHERE username=? AND password_hash=?",(u,hash_pw(p)));r=c.fetchone();conn.close();return r

def logout(): st.session_state.clear(); st.rerun()

# ---------------- AUTH UI ----------------
st.sidebar.title("🧠 AI KYC Verification System")
if "logged_in" not in st.session_state: st.session_state["logged_in"]=False

def auth_ui():
    st.sidebar.subheader("Login / Register")
    act = st.sidebar.radio("Select", ["Login","Register"], key="auth_mode")
    if act=="Login":
        u=st.sidebar.text_input("Username", key="login_u")
        p=st.sidebar.text_input("Password", type="password", key="login_p")
        r=st.sidebar.selectbox("Role", ["User","Admin"], key="login_r")
        if st.sidebar.button("Login", key="login_btn"):
            user=login_user(u,p)
            if user and user[3]==r:
                st.session_state.update({"logged_in":True,"username":u,"role":r})
                st.rerun()
            else: st.sidebar.error("Invalid credentials or role.")
    else:
        u=st.sidebar.text_input("New Username", key="reg_u")
        p=st.sidebar.text_input("Password", type="password", key="reg_p")
        r=st.sidebar.selectbox("Role", ["User","Admin"], key="reg_r")
        if st.sidebar.button("Register", key="reg_btn"):
            st.sidebar.success("Registered!" if register_user(u,p,r) else "Username exists.")

# ---------------- VERIFY SECTIONS ----------------
def verify_section(doc, extractor):
    st.subheader(f"🔹 {doc.title()} Verification")
    name = st.text_input(f"Full Name (as per {doc.title()})", key=f"{doc}_name")
    file = st.file_uploader(f"Upload {doc.title()} Image", type=["jpg","jpeg","png"], key=f"{doc}_file")
    if file and st.button(f"🔍 Verify {doc.title()}", key=f"{doc}_verify"):
        with st.spinner("Analyzing document..."):
            b = file.getvalue()
            text = ocr_text(b)
            number = extractor(text)
            ns = name_similarity(name, text)
            ela = ela_score(b); blur = blur_score(b)
            valid = bool(number)
            fs = fraud_score(valid, ns, ela, blur)
            risk = risk_label(fs)
            rec = {
                "username": st.session_state["username"], "document_type": doc,
                "name_claimed": name, "extracted_number": number,
                "extracted_name": name, "ela_score": ela, "blur_score": blur,
                "name_score": ns, "fraud_score": fs, "risk_level": risk, "ocr_text": text
            }
            save_record(rec)
            st.success("✅ Verification Completed!")
            st.write(f"**Extracted Number:** {mask_number(number)}")
            st.write(f"**Name Match Score:** {ns:.1f}%")
            st.write(f"**Fraud Score:** {fs:.1f}% — {risk}")
            st.text_area("Extracted OCR Text", value=text[:800], height=150)
            st.progress(int(fs))

# ---------------- FINAL CROSS CHECK ----------------
def cross_check(df):
    st.header("🔍 Final Cross Verification (Aadhaar, PAN, License)")
    if df.shape[0] < 3:
        st.info("Please verify all 3 documents first.")
        return
    a = df[df["document_type"]=="aadhaar"].tail(1)
    p = df[df["document_type"]=="pan"].tail(1)
    l = df[df["document_type"]=="license"].tail(1)
    a_name, p_name, l_name = a["name_claimed"].values[0], p["name_claimed"].values[0], l["name_claimed"].values[0]
    n1, n2, n3 = name_similarity(a_name, p_name), name_similarity(p_name, l_name), name_similarity(a_name, l_name)
    avg = (n1+n2+n3)/3
    if avg >= 85: res="✅ Original - All Match"
    elif avg >= 60: res="⚠️ Partial Match"
    else: res="❌ Fake / Mismatch"
    st.subheader(f"Final Result: {res}")
    st.write(f"🪪 Aadhaar Name: **{a_name}**")
    st.write(f"💳 PAN Name: **{p_name}**")
    st.write(f"🚗 License Name: **{l_name}**")
    st.write(f"**Cross Match Score:** {avg:.1f}%")
    fig, ax = plt.subplots()
    ax.bar(["A↔P","P↔L","A↔L"], [n1,n2,n3])
    ax.set_ylim(0,100)
    ax.set_title("Cross-Document Name Similarity (%)")
    st.pyplot(fig)

# ---------------- DASHBOARDS ----------------
def user_dashboard():
    st.title("👤 User Dashboard")
    st.sidebar.button("🚪 Logout", on_click=logout)
    verify_section("aadhaar", extract_aadhaar)
    verify_section("pan", extract_pan)
    verify_section("license", extract_license)
    st.divider()
    df = pd.read_sql_query("SELECT * FROM verifications WHERE username=?", sqlite3.connect(DB_PATH), params=(st.session_state["username"],))
    cross_check(df)
    if not df.empty:
        st.subheader("📊 Your Records")
        st.dataframe(df.tail(10))
        verified = df["extracted_number"].apply(lambda x: 1 if x else 0).sum()
        fig, ax = plt.subplots()
        ax.pie([verified, len(df)-verified], labels=["Verified","Unverified"], autopct="%1.1f%%", startangle=90)
        ax.set_title("Verification Status Distribution")
        st.pyplot(fig)

def admin_dashboard():
    st.title("🛡️ Admin Dashboard")
    st.sidebar.button("🚪 Logout", on_click=logout)
    df = pd.read_sql_query("SELECT * FROM verifications", sqlite3.connect(DB_PATH))
    if df.empty:
        st.info("No records yet.")
        return

    total = len(df)
    approved = sum(df["approval_status"] == "Approved")
    rejected = sum(df["approval_status"] == "Rejected")
    high_risk = round(sum(df["risk_level"].str.contains("High")) / total * 100, 1)

    st.metric("Total Verifications", total)
    st.metric("Approved", approved)
    st.metric("Rejected", rejected)
    st.metric("High Risk (%)", f"{high_risk}%")

    st.dataframe(df)

    id_sel = st.number_input("Enter Record ID to Approve/Reject/Delete", min_value=0, step=1)
    col1, col2, col3 = st.columns(3)
    if col1.button("✅ Approve"):
        update_status(id_sel, "Approved")
        st.success("Record Approved!")
        st.rerun()
    if col2.button("❌ Reject"):
        update_status(id_sel, "Rejected")
        st.warning("Record Rejected!")
        st.rerun()
    if col3.button("🗑️ Delete"):
        conn = sqlite3.connect(DB_PATH)
        conn.execute("DELETE FROM verifications WHERE id=?", (id_sel,))
        conn.commit()
        conn.close()
        st.error("Record Deleted!")
        st.rerun()

    st.download_button("📥 Download CSV", df.to_csv(index=False).encode("utf-8"), "kyc_records.csv")

    col_a, col_b = st.columns(2)
    with col_a:
        fig, ax = plt.subplots()
        df["document_type"].value_counts().plot(kind="pie", autopct="%1.1f%%", ax=ax)
        ax.set_ylabel("")
        ax.set_title("Documents Distribution")
        st.pyplot(fig)

    with col_b:
        fig2, ax2 = plt.subplots()
        df["approval_status"].value_counts().plot(kind="bar", ax=ax2)
        ax2.set_title("Approval vs Rejection Count")
        st.pyplot(fig2)

# ---------------- MAIN ----------------
if not st.session_state["logged_in"]:
    auth_ui()
else:
    if st.session_state["role"]=="Admin":
        admin_dashboard()
    else:
        user_dashboard()


Writing app.py


In [ ]:
from pyngrok import ngrok
import threading, os

!ngrok config add-authtoken "36Qu9bgc0UlJm9PIWRmYG63fVVd_4BrsHqQFcornADrZcgR1t"

port = 8501
public_url = ngrok.connect(port).public_url
print(f"🌍 Your App is Live Here: {public_url}")

def run_app():
    os.system("streamlit run app.py --server.port 8501")

thread = threading.Thread(target=run_app)
thread.start()


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
🌍 Your App is Live Here: https://c103bc9ca19e.ngrok-free.app
